In [13]:
import pandas as pd
from nltk.tokenize import PunktSentenceTokenizer

In [2]:
def show(x):
    print(x.shape)
    return x.head()

In [3]:
citations = pd.read_csv("../data/citations.csv")
show(citations)

(51200, 6)


,paper_id,internal_reference_id,reference_id,context,start_offset,end_offset
0,0,14,15388,e DBLP dataset [19] provides bibliographic me...,16,20
1,0,17,14739,e Cora Information Extraction dataset [22] is...,39,43
2,0,0,18841,e UMass Citation Field Extraction Dataset [1]...,43,46
3,0,15,22352,[20] compiled a dataset consisting of metadata...,0,4
4,0,4,240,e CiteSeerX dataset [6] provides full bibliog...,21,24


In [4]:
references = pd.read_csv("../data/references.csv")
show(references)

(24701, 4)


,reference_id,title,original_citations,abstract
0,0,BERT: Pre-training of Deep Bidirectional Trans...,150,We introduce a new language representation mod...
1,1,Distributed Representations of Words and Phras...,147,The recently introduced continuous Skip-gram m...
2,2,Attention Is All You Need,136,The dominant sequence transduction models are ...
3,3,GloVe : Global Vectors for Word Representation,134,NaN
4,4,Adam: A Method for Stochastic Optimization,111,"We introduce Adam, an algorithm for first-orde..."


In [5]:
papers = pd.read_csv("../data/papers.csv")
show(papers)

(1013, 4)


,paper_id,title,abstract,text
0,0,A Benchmark and Evaluation for Text Extraction...,Extracting the body text from a PDF document i...,A Benchmark and Evaluation for Text Extraction...
1,1,A Benchmark for Systematic Generalization in G...,Humans easily interpret expressions that descr...,Human language is a fabulous tool for generali...
2,2,Ablation Studies in Artificial Neural Networks,Ablation studies have been widely used in the ...,Recent research on deep learning (DL) has brou...
3,3,A brief introduction to weakly supervised lear...,Supervised learning techniques construct predi...,National Key Laboratory for Novel Software Tec...
4,4,Abstractive Event Summarization on Twitter,ive Event Summarization on Twitter Quanzhi Li ...,• Computing methodologies → Natural language p...


# the most mentioned papers

In [6]:
citations.groupby(["paper_id", "reference_id"]).size()\
.sort_values(ascending = False)\
.rename("mentions")\
.reset_index()\
.head(20)

,paper_id,reference_id,mentions
0,481,3022,105
1,540,1840,51
2,15,1637,42
3,181,161,41
4,633,25,34
5,697,927,30
6,697,726,28
7,633,49,27
8,625,23174,27
9,354,1870,26


In [7]:
citations.groupby(["reference_id", "paper_id"]).size()\
.rename("mentions")\
.reset_index()\
.sort_values("mentions", ascending = False)\
.groupby("reference_id")\
.mentions.max()\
.head(20)\
.reset_index()

,reference_id,mentions
0,0,14
1,1,14
2,2,7
3,3,10
4,4,4
5,5,5
6,6,7
7,7,10
8,8,8
9,9,6


In [8]:
task_pool = citations.groupby(["reference_id", "paper_id"]).size()\
.rename("mentions")\
.reset_index()\
.sort_values("mentions", ascending = False)\
.groupby("reference_id")\
.first()\
.head(20)\
.reset_index()

task_pool

,reference_id,paper_id,mentions
0,0,754,14
1,1,783,14
2,2,775,7
3,3,773,10
4,4,223,4
5,5,43,5
6,6,961,7
7,7,773,10
8,8,773,8
9,9,281,6


#  training sentence tokenizer

In [11]:
papers.text.map(len).sum()

39889755

In [10]:
len(papers.text.sum())

39889755

In [14]:
tokenizer = PunktSentenceTokenizer(papers.text.sum())

In [18]:
task_pool.merge(papers[["paper_id", "text"]])\
.assign(sentences = lambda df: df.text.map(tokenizer.tokenize))\
.assign(n_sentences = lambda df: df.sentences.map(len))

,reference_id,paper_id,mentions,text,sentences,n_sentences
0,0,754,14,ar X iv :1 90 7. 11 69 2v 1 [ cs .C L ] 2 6 Ju...,"[ar X iv :1 90 7., 11 69 2v 1 [ cs .C L ] 2 6 ...",267
1,1,783,14,List of Abbreviations and Symbols Used . . . ....,"[List of Abbreviations and Symbols Used ., ., ...",6584
2,2,775,7,Humans and natural phenomena often generate a ...,[Humans and natural phenomena often generate a...,329
3,3,773,10,Contents lists available at ScienceDirect Jour...,[Contents lists available at ScienceDirect Jou...,504
4,7,773,10,Contents lists available at ScienceDirect Jour...,[Contents lists available at ScienceDirect Jou...,504
5,8,773,8,Contents lists available at ScienceDirect Jour...,[Contents lists available at ScienceDirect Jou...,504
6,4,223,4,SUB-WORD EMBEDDINGS by Mihir Joshi Submitted i...,[SUB-WORD EMBEDDINGS by Mihir Joshi Submitted ...,2082
7,5,43,5,Proceedings of the 27th International Conferen...,[Proceedings of the 27th International Confere...,275
8,6,961,7,We live in a world where most of the informati...,[We live in a world where most of the informat...,237
9,9,281,6,0 DeepTracker: Visualizing the Training Proces...,[0 DeepTracker: Visualizing the Training Proce...,678
